In [ ]:
import sys
sys.path.append(".../RADAR_Plataform/RADAR") # dir library

# 🆕 **Example: adding a new model to FlexAnomalyDetection **

This notebook shows how to **add a new model** to the library by defining it externally.  

In this example, we show how to add a model using the **`FlexAnomalyDetection`** class to join it to the federated data models. 

We define the **new model** along with its **related parameters**. For demonstration purposes, we create a **very simple and basic model** in this example.

In [ ]:
from RADAR.federated_data.algorithms import flexanomalies
from RADAR.static_data.static_datasets_uci import global_load
from sklearn.model_selection import train_test_split

Load example dataset, dataset from the static data dataset folder 

In [4]:
X,y = global_load('default_of_credit_card_clients')   #name dataset in static datasets uci repo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

Metadata: {'uci_id': 350, 'name': 'Default of Credit Card Clients', 'repository_url': 'https://archive.ics.uci.edu/dataset/350/default+of+credit+card+clients', 'data_url': 'https://archive.ics.uci.edu/static/public/350/data.csv', 'abstract': "This research aimed at the case of customers' default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods.", 'area': 'Business', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 30000, 'num_features': 23, 'feature_types': ['Integer', 'Real'], 'demographics': ['Sex', 'Education Level', 'Marital Status', 'Age'], 'target_col': ['Y'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2009, 'last_updated': 'Fri Mar 29 2024', 'dataset_doi': '10.24432/C55S3H', 'creators': ['I-Cheng Yeh'], 'intro_paper': {'ID': 365, 'type': 'NATIVE', 'title': 'The comparisons of data mining techniques for the predictive ac

####  **Note on the Model**

The following model is a **simple example** created for demonstration purposes.  
It is not intended for production use. Its purpose is to illustrate how to define a custom anomaly **MyAnomalyModel** detection model, integrate it with `FlexAnomalyDetection`, and use it with federated or centralized data.

In [24]:
class MyAnomalyModel:
    def __init__(self, param1=1, param2=2):
        """
        Simple anomaly detection model.
        
        Parameters:
        -----------
        param1 : int or float, optional
            Example parameter 1
        param2 : int or float, optional
            Example parameter 2
        """
        self.param1 = param1
        self.param2 = param2
        self.mean_ = None
        self.std_ = None

    def fit(self, X, y=None):
        import numpy as np
        self.mean_ = np.mean(X, axis=0)
        self.std_ = np.std(X, axis=0)
        print(f"Fitting model with param1={self.param1}, param2={self.param2}")
        return self

    def predict(self, X):
        import numpy as np
        if self.mean_ is None or self.std_ is None:
            raise ValueError("Model has not been fitted yet.")
        lower = self.mean_ - 2 * self.std_
        upper = self.mean_ + 2 * self.std_
        anomalies = np.any((X < lower) | (X > upper), axis=1)
        return anomalies


You can now instantiate using the new algorithm

In [25]:
flexanomalies.FlexAnomalyDetection.register_algorithm("myAnomalyModel", MyAnomalyModel)
detector = flexanomalies.FlexAnomalyDetection(algorithm_="myAnomalyModel",param1=10, param2=20)

 The algorithm myAnomalyModel is already registered and will be overwritten.
Federated Params:{} 
 Model Params:{'algorithm_': 'myAnomalyModel', 'param1': 10, 'param2': 20}


Define new federated functions 

In [18]:

detector.register_federated_functions("customAnomaly", {  #  Define federated functions 
    "build_model": lambda: print("Building Custom Model"),
    "copy": lambda: print("Copying Custom Model"),
    "train": lambda: print("Training Custom Model"),
    "collect": lambda: print("Collecting Custom Model Weights"),
    "aggregate": lambda: print("Aggregating Custom Model Weights"),
    "set_weights": lambda: print("Setting Aggregated Weights"),
})

 The algorithm myAnomalyModel is already registered and will be overwritten.
Federated Params:{} 
 Model Params:{'algorithm_': 'myAnomalyModel', 'param1': 10, 'param2': 20}


In [26]:
#Example using a federated external data set but using flex dataset format 
from flexanomalies.utils.load_data import federate_data
detector.flex_data = federate_data(5, X_train, y_train)

In [ ]:
# detector.fit(X_train,y_train)

# # Hacemos predicciones sobre datos de prueba
# anomalies = detector.predict(X_test)

# print("Predicted anomalies:", anomalies)

# 🆕 Another Example: adding a new model to PyodAnomalyDetection

This section shows how to add a new anomaly detection model to the library by defining it externally and registering it in PyodAnomalyDetection.

In this example, we demonstrate how to add a model using the PyodAnomalyDetection class. We define a new custom model along with its related parameters. For demonstration purposes, we use a very simple and basic anomaly detecto

In [5]:
from RADAR.static_data.preprocessing.preprocessing_static import StandardScalerPreprocessing 
from RADAR.static_data.algorithms import pyod

In [6]:
X,y = global_load('default_of_credit_card_clients')   #name dataset in static datasets uci repo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

Metadata: {'uci_id': 350, 'name': 'Default of Credit Card Clients', 'repository_url': 'https://archive.ics.uci.edu/dataset/350/default+of+credit+card+clients', 'data_url': 'https://archive.ics.uci.edu/static/public/350/data.csv', 'abstract': "This research aimed at the case of customers' default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods.", 'area': 'Business', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 30000, 'num_features': 23, 'feature_types': ['Integer', 'Real'], 'demographics': ['Sex', 'Education Level', 'Marital Status', 'Age'], 'target_col': ['Y'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2009, 'last_updated': 'Fri Mar 29 2024', 'dataset_doi': '10.24432/C55S3H', 'creators': ['I-Cheng Yeh'], 'intro_paper': {'ID': 365, 'type': 'NATIVE', 'title': 'The comparisons of data mining techniques for the predictive ac

In [9]:
from sklearn.base import BaseEstimator
import numpy as np

# Ejemplo de un modelo simple de PyOD
class MyCustomAnomalyDetector(BaseEstimator):
    def __init__(self, threshold=0.5):
        self.threshold = threshold

    def fit(self, X, y=None):
        # Save the average as a “simple model”
        self.mean_ = np.mean(X, axis=0)
        return self

    def decision_function(self, X):
        # Simple score: Euclidean distance to the mean
        return np.linalg.norm(X - self.mean_, axis=1)

    def predict(self, X):
        # 1 anomaly , 0 normal
        scores = self.decision_function(X)
        return (scores > self.threshold).astype(int)

In [16]:
#Preprocess Data
scaler = StandardScalerPreprocessing()
X_scaled = scaler.fit_transform(X_train)


In [17]:
pyod.PyodAnomalyDetection.register_algorithm("my_custom", MyCustomAnomalyDetector)

 The algorithm my_custom is already registered and will be overwritten.
Algorithm my_custom registered successfully.


In [18]:
kwargs = {"algorithm_": "my_custom", "threshold":1.0}
model_new = pyod.PyodAnomalyDetection(**kwargs)
print(model_new.get_params())

{'label_parser': None, 'algorithm_': 'MyCustomAnomalyDetector', 'threshold': 1.0}


In [19]:
model_new.fit(X_scaled)

In [20]:
Xtest_scaled = scaler.fit_transform(X_test)
pred = model_new.predict(Xtest_scaled)
print("Prediction:",pred)

Prediction: [1 1 1 ... 1 1 1]
